## COD API, One Match --> explore API, format, clean & reshape

## import stuff to work with

In [26]:
import asyncio
import os
import sys
import dotenv
from pprint import pprint
import datetime
from datetime import datetime, timezone, timedelta
import pandas as pd
import numpy as np
import pickle
import altair as alt

import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType, Language

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

In [3]:
# we're storing our string conversion module above current working (notebooks) directory
sys.path.insert(0, os.path.abspath('../wzkd'))
import labels
from labels import mode_labels

In [4]:
import urllib.parse
from typing import List, Optional, Union

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request

async def GetMatchStats(
    self, platform, title: Title, mode: Mode, matchId: int, language: Language = Language.English, **kwargs
):
    """ 
    Compared to client : modified so that we do not use Platform.abc as parameter
    but instead our app-defined workflow (drop down menu) to select our platform of choice"
    """
    return (
        await self.http.GetFullMatch(
            title.value, platform, mode.value, matchId, language.value
        )
    )["data"]["allPlayers"]
    # api result, at very least for Warzone {'data':{'all_players:' is the only key},'status': call status}

Client.GetMatchStats = GetMatchStats
match = await client.GetMatchStats('battle', Title.ModernWarfare, Mode.Warzone, matchId=3670970687107920841)

# example of IDs :
# 11672696746036290501 a "custom" mode game (Rumble Clash)
# 6825832239054239925 : Iron Trials Trios
# 9207377216339591388 : BR Duo

In [5]:
#save file if you want to work in offline mode
#with open("match.pkl", 'wb') as f:
#    pickle.dump(match, f)

### Overview of returned (Match) stats

In [6]:
# load offline data
open_file = open("match.pkl", "rb")
match = pickle.load(open_file)
open_file.close()

In [7]:
pprint(match[0:2], depth=5)

[{'draw': False,
  'duration': 1593000,
  'gameType': 'wz',
  'map': 'mp_don4',
  'matchID': '3670970687107920841',
  'mode': 'br_brquads',
  'player': {'awards': {},
             'brMissionStats': {'missionStatsByType': {'scavenger': {...}},
                                'missionsComplete': 2,
                                'totalMissionWeaponXpEarned': 420.0,
                                'totalMissionXpEarned': 1200.0},
             'clantag': '^3lkf22^7',
             'loadout': [{'extraPerks': [...],
                          'killstreaks': [...],
                          'lethal': {...},
                          'perks': [...],
                          'primaryWeapon': {...},
                          'secondaryWeapon': {...},
                          'tactical': {...}},
                         {'extraPerks': [...],
                          'killstreaks': [...],
                          'lethal': {...},
                          'perks': [...],
                       

In [8]:
print(match[0]['player'].keys(), "\n")
pprint(match[0]['player'], depth=3)

dict_keys(['team', 'rank', 'awards', 'username', 'uno', 'clantag', 'brMissionStats', 'loadout']) 

{'awards': {},
 'brMissionStats': {'missionStatsByType': {'scavenger': {...}},
                    'missionsComplete': 2,
                    'totalMissionWeaponXpEarned': 420.0,
                    'totalMissionXpEarned': 1200.0},
 'clantag': '^3lkf22^7',
 'loadout': [{'extraPerks': [...],
              'killstreaks': [...],
              'lethal': {...},
              'perks': [...],
              'primaryWeapon': {...},
              'secondaryWeapon': {...},
              'tactical': {...}},
             {'extraPerks': [...],
              'killstreaks': [...],
              'lethal': {...},
              'perks': [...],
              'primaryWeapon': {...},
              'secondaryWeapon': {...},
              'tactical': {...}}],
 'rank': 54.0,
 'team': 'team_eighteen',
 'uno': '13831131263765265947',
 'username': 'Confetti_Seeker'}


#### Extract Teammates

In [9]:
def extract_teammates(match, username):
    """ Explore down match, team/username keys, and retrive teammates of a given player"""
    team = [player['player']['team'] for player in match if player['player']['username'] == username][0]
    teammates = [player['player']['username'] for player in match if player['player']['team'] == team]
    return teammates

In [10]:
teammates = extract_teammates(match, "gentil_renard")
teammates

['Confetti_Seeker', 'Marmiton', 'gentil_renard', 'Moinolol']

In [11]:
df_match = pd.DataFrame(match)
df_match.head(3)

,utcStartSeconds,utcEndSeconds,map,mode,matchID,duration,playlistName,version,gameType,playerCount,playerStats,player,teamCount,rankedTeams,draw,privateMatch
0,1635974372,1635975965,mp_don4,br_brquads,3670970687107920841,1593000,None,1,wz,152,"{'kills': 2.0, 'medalXp': 30.0, 'objectiveLast...","{'team': 'team_eighteen', 'rank': 54.0, 'award...",39,None,False,False
1,1635974372,1635975965,mp_don4,br_brquads,3670970687107920841,1593000,None,1,wz,152,"{'kills': 0.0, 'medalXp': 0.0, 'matchXp': 557....","{'team': 'team_fourteen', 'rank': 54.0, 'award...",39,None,False,False
2,1635974372,1635975965,mp_don4,br_brquads,3670970687107920841,1593000,None,1,wz,152,"{'kills': 1.0, 'medalXp': 45.0, 'objectiveLast...","{'team': 'team_nineteen', 'rank': 54.0, 'award...",39,None,False,False


#### Functions

In [12]:
def MatchPlayersToDf(match):
    """
    COD API / callofduty.py client --> list of dict : +- 150 players with their stats in a given match
    Share similar strucure (keys) to Matches result, thus very similar operations to what's performed in MatchesToDf()
    We expand Player and PlayerStats levels, concatenate them
    ! We drop some cols we dont want to work with; while having 2 new cols 'team' and 'username' (compared to Matches result)
    
    Returns
    -------
    DataFrame, a match stats, players as rows, matches/player stats as columns
    """
    
    keep_cols =  [
        'mode',
        'utcEndSeconds',
        'team',
        'username',
        'timePlayed',
        'teamPlacement',
        'kdRatio', 
        'kills', 
        'deaths', 
        'assists', 
        'damageDone',
        'damageTaken',
        'gulagKills',
        'percentTimeMoving',
        'duration'
        ]

    df = pd.DataFrame(match)
    
    # column playerStats is a series of dict, we can expand it easily and append, then drop the original
    df = pd.concat([df.drop(['playerStats'], axis=1), df['playerStats'].apply(pd.Series)], axis=1)
    
    # colum player has more depth
    # once expanded, it has a column 'loadout' : a series of list of dict (either one or more, we will keep 1 to max 3)
    # and also brMissionStats (mostly empty ?) that we aren't interested in
    
    df = pd.concat([df.drop(['player'], axis=1), df['player'].apply(pd.Series)], axis=1)
    df = df.drop(['brMissionStats'], axis = 1)
    df = pd.concat([df.drop(['loadout'], axis=1), df['loadout'].apply(pd.Series)], axis=1)
    for col in range(0,3):
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x['primaryWeapon']['name']} - {x['secondaryWeapon']['name']}" if not str(x) == 'nan' else np.nan)
            col_name = "loadout_" + str(col +1)
            df = df.rename(columns={col: f"loadout_{str(col +1)}"})   
            keep_cols.append(col_name)
    
    return df[keep_cols]

In [13]:
open_file = open("match.pkl", "rb")
match = pickle.load(open_file)
open_file.close()

In [14]:
df_match = MatchPlayersToDf(match)
df_match.head(3)

,mode,utcEndSeconds,team,username,timePlayed,teamPlacement,kdRatio,kills,deaths,assists,damageDone,damageTaken,gulagKills,percentTimeMoving,duration,loadout_1,loadout_2,loadout_3
0,br_brquads,1635975965,team_eighteen,Confetti_Seeker,478.0,30.0,1.000000,2.0,2.0,0.0,601.0,438.0,1.0,73.22654,1593000,iw8_sm_t9fastfire - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,NaN
1,br_brquads,1635975965,team_fourteen,Soros Gyorgy,168.0,37.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,77.77778,1593000,iw8_ar_t9standard - iw8_sm_t9fastfire,NaN,NaN
2,br_brquads,1635975965,team_nineteen,riderskunk,934.0,17.0,0.333333,1.0,3.0,0.0,465.0,606.0,0.0,58.75441,1593000,iw8_ar_t9standard - iw8_sm_t9fastfire,iw8_pi_t9semiauto - iw8_fists,iw8_sh_t9pump - iw8_fists


In [15]:
def MatchPlayersStandardize(df):
    """
    A first layer of standadization (as properly formated) to our a "Match with players stats" DataFrame
    For further aggregations / better readibility of our data
    
    Returns
    -------
    DataFrame : players/teams as rows, cleaned player stats of a given match as columns
    """
    
    int_cols =  [
        'teamPlacement', 
        'kills', 
        'deaths', 
        'assists', 
        'gulagKills', 
        'damageDone',
        'damageTaken'
        ]
    
    float_cols = [
        'kdRatio',
        'percentTimeMoving'
        ]
    
    ts_cols = [
        'utcEndSeconds'
        ]
    
    columns_labels = {
        'utcEndSeconds':'Ended at',
        'timePlayed': 'Playtime',
        'teamPlacement':'#',
        'kdRatio':'KD',
        'damageDone':'Damage >',
        'damageTaken':'Damage <',
        'gulagKills':'Gulag',
        'headshots':'% headshots',
        'percentTimeMoving':'% moving',
        'duration':'Game duration'
        }

    # df = df.fillna(0)
    df[int_cols] = df[int_cols].astype(int)
    df[float_cols] = df[float_cols].astype(float).round(1) # still renders 0.0000 in streamlit but ugly hacks exists
    
    # specials
    df['utcEndSeconds'] = df['utcEndSeconds'].apply(lambda x: datetime.fromtimestamp(x))
    
    df['duration'] = df['duration'].apply(lambda x: x/1000).apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M')) # API duration is in seconds x1000
    df['timePlayed'] = df['timePlayed'].apply(lambda x: pd.to_datetime(x, unit='s').strftime('%M:%S')) # API timePlayed is in seconds
    df['gulagKills'] = df['gulagKills'].map({1:'W', 0:'L'})
    for col in ['loadout_1', 'loadout_2', 'loadout_3']:
            df.fillna({col:'-'}, inplace=True) if col in df.columns else None
    
    df = df.replace({"mode": mode_labels})
    df = df.rename(columns=columns_labels)
    df.columns = df.columns.str.capitalize()
    df = df.rename({"Kd":"KD"}, axis=1)
     
    return df

In [16]:
open_file = open("match.pkl", "rb")
match = pickle.load(open_file)
open_file.close()
df_match = MatchPlayersToDf(match)
df = MatchPlayersStandardize(df_match)
df.head()

,Mode,Ended at,Team,Username,Playtime,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,% moving,Game duration,Loadout_1,Loadout_2,Loadout_3
0,Quads,2021-11-03 22:46:05,team_eighteen,Confetti_Seeker,07:58,30,1.0,2,2,0,601,438,W,73.2,26,iw8_sm_t9fastfire - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,-
1,Quads,2021-11-03 22:46:05,team_fourteen,Soros Gyorgy,02:48,37,0.0,0,0,0,0,0,L,77.8,26,iw8_ar_t9standard - iw8_sm_t9fastfire,-,-
2,Quads,2021-11-03 22:46:05,team_nineteen,riderskunk,15:34,17,0.3,1,3,0,465,606,L,58.8,26,iw8_ar_t9standard - iw8_sm_t9fastfire,iw8_pi_t9semiauto - iw8_fists,iw8_sh_t9pump - iw8_fists
3,Quads,2021-11-03 22:46:05,team_twenty_four,GinTo,10:31,27,0.5,1,2,0,125,539,L,82.3,26,iw8_sm_victor - iw8_knife,iw8_pi_t9semiauto - iw8_fists,-
4,Quads,2021-11-03 22:46:05,team_three,iSoniK_TV,24:15,5,2.0,6,3,3,4173,803,W,88.3,26,iw8_ar_t9british - iw8_sm_t9cqb,iw8_pi_t9semiauto - iw8_fists,-


In [17]:
## add aggregations by teams

#### Some visualizations

In [54]:
open_file = open("match.pkl", "rb")
match = pickle.load(open_file)
open_file.close()
df_match = MatchPlayersToDf(match)
df = MatchPlayersStandardize(df_match)
df.head(2)

,Mode,Ended at,Team,Username,Playtime,#,KD,Kills,Deaths,Assists,Damage >,Damage <,Gulag,% moving,Game duration,Loadout_1,Loadout_2,Loadout_3
0,Quads,2021-11-03 22:46:05,team_eighteen,Confetti_Seeker,07:58,30,1.0,2,2,0,601,438,W,73.2,26,iw8_sm_t9fastfire - iw8_sn_t9accurate,iw8_pi_t9semiauto - iw8_fists,-
1,Quads,2021-11-03 22:46:05,team_fourteen,Soros Gyorgy,02:48,37,0.0,0,0,0,0,0,L,77.8,26,iw8_ar_t9standard - iw8_sm_t9fastfire,-,-


In [52]:
base = alt.Chart(df)         
hist2 = base.mark_bar().encode(
    x=alt.X('Kills:Q', bin=alt.BinParams(step=1)),
    y=alt.Y('count()', axis=alt.Axis(format='', title='n Players')),
    tooltip=['Kills'],
    color=alt.value("orange")

).properties(width=400, height=300)         
red_median_line = base.mark_rule(color='red').encode(
    x=alt.X('median(Kills):Q', title='Kills'),
    size=alt.value(3)
)
hist2 + red_median_line

alt.LayerChart(...)

In [50]:
base = alt.Chart(df)

hist2 = base.mark_bar().encode(
    x=alt.X('Kills:Q', bin=alt.BinParams(step=1)),
    y=alt.Y('count()', axis=alt.Axis(tickCount=10, format='', title='n Players')),
    tooltip=['Kills', 'count()'],
    color=alt.value("orange")
            
).properties(width=400, height=200)

red_median_line = base.mark_rule(color='black').encode(
    x=alt.X('median(KD):Q', title='KD'),
    size=alt.value(2)
)

hist2 + red_median_line

alt.LayerChart(...)

In [48]:
hist2 = base.mark_bar().encode(
    x=alt.X('KD:Q', bin=alt.Bin(step=0.5)), 
    y=alt.Y('count()', axis=alt.Axis(format='', title='n Players')),
    tooltip=['KD', 'count()'],
    color=alt.value("orange")
            
).properties(width=400, height=300)

red_median_line2 = base.mark_rule(color='black').encode(
    x=alt.X('median(KD):Q', title='KD'),
    size=alt.value(2)
)
hist2 + red_median_line2

alt.LayerChart(...)

In [ ]:
alt.Chart(source).mark_tick().encode(
    x='Kills:Q',
    y='Cylinders:O'
)